In [2]:
import pandas as pd
old_data = pd.read_csv("../data/listings_detailed.csv")
new_data = pd.read_csv("../data/listings.csv")

In [ ]:
old_data.shape, new_data.shape
'id' in old_data.columns, 'id' in new_data.columns
old_data['id'] = old_data['id'].astype(str)
new_data['id'] = new_data['id'].astype(str)

old_data.columns.tolist()
old_data.head(2)
print(old_data['id'][0])

<class 'str'>


In [16]:
new_ids = set(new_data['id'])
old_data['survived'] = old_data['id'].apply(lambda x: 1 if x in new_ids else 0)
old_data['survived'].value_counts(), old_data['survived'].value_counts(normalize=True)

(survived
 1    9105
 0    5263
 Name: count, dtype: int64,
 survived
 1    0.6337
 0    0.3663
 Name: proportion, dtype: float64)

In [18]:
old_data.columns.tolist()

['id',
 'listing_url',
 'scrape_id',
 'last_scraped',
 'source',
 'name',
 'description',
 'neighborhood_overview',
 'picture_url',
 'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'latitude',
 'longitude',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bathrooms_text',
 'bedrooms',
 'beds',
 'amenities',
 'price',
 'minimum_nights',
 'maximum_nights',
 'minimum_minimum_nights',
 'maximum_minimum_nights',
 'minimum_maximum_nights',
 'maximum_maximum_nights',
 'minimum_nights_avg_ntm',
 'maximum_nights_avg_ntm',
 'calendar_updated',
 'has_availability',
 'availability_30

In [41]:
df = old_data.copy()

# Keep columns that might be useful for modeling
useful_columns = [
    'id',
    'host_is_superhost',
    'host_response_time',
    'host_response_rate',
    'host_acceptance_rate',
    'host_total_listings_count',
    'host_has_profile_pic',
    'host_since',
    'property_type',
    'room_type',
    'accommodates',
    'bathrooms_text',
    'minimum_nights',
    'maximum_nights',
    'instant_bookable',
    'availability_365',
    'price',
    'number_of_reviews',
    'reviews_per_month',
    'review_scores_rating',
    'review_scores_accuracy',
    'review_scores_cleanliness',
    'review_scores_checkin',
    'review_scores_communication',
    'review_scores_location',
    'review_scores_value',
    'amenities',
    'neighbourhood_cleansed',
    'survived',
]
df = df[useful_columns]
# Now we have the columns that will be useful for modeling

In [42]:
print(df['host_is_superhost'].value_counts(dropna=False))


host_is_superhost
f      9406
t      4957
NaN       5
Name: count, dtype: int64


In [44]:
# Convert 't'/'f' to 1/0
df['host_has_profile_pic'] = df['host_has_profile_pic'].map({'t': 1, 'f': 0})
df['instant_bookable'] = df['instant_bookable'].map({'t': 1, 'f': 0})
df['host_is_superhost'] = df['host_is_superhost'].map({'t': 1, 'f': 0})
df.head(20)

,id,host_is_superhost,host_response_time,host_response_rate,host_acceptance_rate,host_total_listings_count,host_has_profile_pic,host_since,property_type,room_type,...,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,amenities,neighbourhood_cleansed,survived
0,5456,1.0,within an hour,100%,95%,4.0,1.0,2009-02-16,Entire guesthouse,Entire home/apt,...,4.84,4.88,4.85,4.89,4.82,4.73,4.79,"[""Heating"", ""Backyard"", ""Bed linens"", ""Hot wat...",78702,1
1,5769,1.0,within a few hours,100%,95%,4.0,1.0,2009-02-19,Private room in home,Private room,...,4.90,4.90,4.86,4.91,4.94,4.76,4.92,"[""Private backyard"", ""Free parking on premises...",78729,1
2,6413,1.0,NaN,NaN,100%,1.0,1.0,2009-04-17,Entire guesthouse,Entire home/apt,...,4.97,4.99,4.99,4.99,4.98,4.87,4.93,"[""Heating"", ""Outdoor dining area"", ""Dove body ...",78704,1
3,6448,1.0,within an hour,100%,100%,2.0,1.0,2009-04-20,Entire guesthouse,Entire home/apt,...,4.97,4.97,4.96,4.99,4.97,4.97,4.90,"[""Pack \u2019n play/Travel crib"", ""Free parkin...",78704,1
4,8502,0.0,within a day,60%,70%,1.0,1.0,2009-07-11,Entire guest suite,Entire home/apt,...,4.53,4.49,4.67,4.83,4.86,4.64,4.57,"[""Essentials"", ""Heating"", ""Central air conditi...",78741,1
5,13035,1.0,within an hour,100%,94%,2.0,1.0,2009-11-02,Entire home,Entire home/apt,...,5.00,4.94,4.94,5.00,5.00,5.00,4.94,"[""Shared backyard \u2013 Fully fenced"", ""Heati...",78702,1
6,18258,0.0,within an hour,100%,100%,4.0,1.0,2009-09-18,Entire bungalow,Entire home/apt,...,5.00,5.00,5.00,5.00,5.00,4.50,4.50,"[""Free parking on premises"", ""Heating"", ""Amazo...",78745,0
7,22828,1.0,NaN,NaN,100%,1.0,1.0,2009-11-22,Entire guesthouse,Entire home/apt,...,4.94,5.00,4.98,5.00,5.00,4.77,4.86,"[""Bed linens"", ""Hot water"", ""Portable fans"", ""...",78741,1
8,22982,1.0,within an hour,100%,98%,5.0,1.0,2010-03-06,Entire guesthouse,Entire home/apt,...,4.91,4.93,4.97,4.96,4.90,4.88,4.82,"[""Free parking on premises"", ""Heating"", ""Bed l...",78703,0
9,23496,1.0,within an hour,100%,95%,4.0,1.0,2009-02-16,Entire home,Entire home/apt,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[""Pantene conditioner"", ""Free parking on premi...",78702,0


In [49]:
def pct_to_float(x):
    if isinstance(x, str) and x.endswith('%'):
        return float(x.replace('%', "")) / 100.0
    return None

df['host_response_rate'] = df['host_response_rate'].apply(pct_to_float)
df['host_acceptance_rate'] = df['host_acceptance_rate'].apply(pct_to_float)
df[['host_response_rate', 'host_acceptance_rate']].head(10)

,host_response_rate,host_acceptance_rate
0,1.0,0.95
1,1.0,0.95
2,NaN,1.00
3,1.0,1.00
4,0.6,0.70
5,1.0,0.94
6,1.0,1.00
7,NaN,1.00
8,1.0,0.98
9,1.0,0.95


In [ ]:
# Fix bathrooms_text to extract numeric bathroom count
df['bathrooms'] = df['bathrooms_text'].str.extract(r'(\d+\.?\d*)')
df['bathrooms'] = df['bathrooms'].astype(float)
df.drop(columns=['bathrooms_text'], inplace=True)


In [ ]:
df[['bathrooms']].head(20)

In [56]:
# Fix price column to be numeric
df['price'] = df['price'].replace('[\$,]', '', regex=True).astype(float)
df['price'].head(10)

0    176.0
1     42.0
2    109.0
3    240.0
4     85.0
5    160.0
6    171.0
7     65.0
8    350.0
9    200.0
Name: price, dtype: float64

In [57]:
# Update host_since to be number of days since hosting started
snapshot_date = pd.to_datetime("2023-03-31")
df['host_since'] = pd.to_datetime(df['host_since'], errors='coerce')
df['host_tenure_days'] = (snapshot_date - df['host_since']).dt.days
df.drop(columns=['host_since'], inplace=True)
df['host_tenure_days'].head(10)

0    5156.0
1    5153.0
2    5096.0
3    5093.0
4    5011.0
5    4897.0
6    4942.0
7    4877.0
8    4773.0
9    5156.0
Name: host_tenure_days, dtype: float64

In [59]:
df['host_tenure_days'].min(), df['host_tenure_days'].max()

(18.0, 5506.0)

In [60]:
df['has_wifi'] = df['amenities'].str.contains('Wifi', case=False, na=False).astype(int)
df['has_kitchen'] = df['amenities'].str.contains('Kitchen', case=False, na=False).astype(int)
df['has_air_conditioning'] = df['amenities'].str.contains('Air conditioning', case=False, na=False).astype(int)
df['has_free_parking'] = df['amenities'].str.contains('Free parking', case=False, na=False).astype(int)
df['has_heating'] = df['amenities'].str.contains('Heating', case=False, na=False).astype(int)
df['has_bed_linens'] = df['amenities'].str.contains('Bed linens', case=False, na=False).astype(int)
df['has_tv'] = df['amenities'].str.contains('TV', case=False, na=False).astype(int)
df.drop(columns=['amenities'], inplace=True)
df['has_wifi'].head(10)

0    1
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    0
Name: has_wifi, dtype: int64

In [61]:
review_cols = [
    'review_scores_rating',
    'review_scores_accuracy',
    'review_scores_cleanliness',
    'review_scores_checkin',
    'review_scores_communication',
    'review_scores_location',
    'review_scores_value',
]
for col in review_cols:
    df[col] = df[col].fillna(df[col].median())



In [63]:
df.to_csv("../data/processed_listings.csv", index=False)

In [64]:
# Fill missing host_response_time, rate, and acceptance rate with unknown/0
df['host_response_time'] = df['host_response_time'].fillna('unknown')

# Create missing flags
df['missing_response_rate'] = df['host_response_rate'].isna().astype(int)
df['missing_acceptance_rate'] = df['host_acceptance_rate'].isna().astype(int)

# Fill missing values with median
df['host_response_rate'] = df['host_response_rate'].fillna(df['host_response_rate'].median())
df['host_acceptance_rate'] = df['host_acceptance_rate'].fillna(df['host_acceptance_rate'].median())


In [74]:
bedrooms_col = old_data[['id', 'beds']]
df = df.merge(bedrooms_col, on='id', how='left')

In [75]:
df['beds'].head(10)

0    2.0
1    1.0
2    1.0
3    2.0
4    1.0
5    2.0
6    2.0
7    1.0
8    2.0
9    4.0
Name: beds, dtype: float64

In [76]:
# Fix missing numeric columns
df['bathrooms'] = df['bathrooms'].fillna(df['bathrooms'].median())
df['bedrooms'] = df['bedrooms'].fillna(df['bedrooms'].median())
df['beds'] = df['beds'].fillna(df['beds'].median())

df['host_total_listings_count'] = df['host_total_listings_count'].fillna(1)
df['host_tenure_days'] = df['host_tenure_days'].fillna(0)
df['host_has_profile_pic'] = df['host_has_profile_pic'].fillna(0)
df['host_is_superhost'] = df['host_is_superhost'].fillna(0)
df.head(20)

,id,host_is_superhost,host_response_time,host_response_rate,host_acceptance_rate,host_total_listings_count,host_has_profile_pic,property_type,room_type,accommodates,...,has_kitchen,has_air_conditioning,has_free_parking,has_heating,has_bed_linens,has_tv,missing_response_rate,missing_acceptance_rate,bedrooms,beds
0,5456,1.0,within an hour,1.0,0.95,4.0,1.0,Entire guesthouse,Entire home/apt,3,...,1,1,0,1,1,1,0,0,1.0,2.0
1,5769,1.0,within a few hours,1.0,0.95,4.0,1.0,Private room in home,Private room,2,...,0,1,1,1,1,1,0,0,1.0,1.0
2,6413,1.0,unknown,1.0,1.00,1.0,1.0,Entire guesthouse,Entire home/apt,2,...,1,1,0,1,1,1,1,0,2.0,1.0
3,6448,1.0,within an hour,1.0,1.00,2.0,1.0,Entire guesthouse,Entire home/apt,2,...,1,1,1,1,1,1,0,0,1.0,2.0
4,8502,0.0,within a day,0.6,0.70,1.0,1.0,Entire guest suite,Entire home/apt,2,...,1,1,0,1,0,0,0,0,1.0,1.0
5,13035,1.0,within an hour,1.0,0.94,2.0,1.0,Entire home,Entire home/apt,3,...,1,1,0,1,1,1,0,0,2.0,2.0
6,18258,0.0,within an hour,1.0,1.00,4.0,1.0,Entire bungalow,Entire home/apt,3,...,1,1,1,1,1,1,0,0,3.0,2.0
7,22828,1.0,unknown,1.0,1.00,1.0,1.0,Entire guesthouse,Entire home/apt,2,...,1,0,0,1,1,1,1,0,1.0,1.0
8,22982,1.0,within an hour,1.0,0.98,5.0,1.0,Entire guesthouse,Entire home/apt,4,...,0,1,1,1,1,1,0,0,2.0,2.0
9,23496,1.0,within an hour,1.0,0.95,4.0,1.0,Entire home,Entire home/apt,6,...,1,1,1,1,1,1,0,0,3.0,4.0


In [77]:
cat_columns = [
    'host_response_time',
    'property_type',
    'room_type',
    'neighbourhood_cleansed',
]
df = pd.get_dummies(df, columns=cat_columns, drop_first=True)

In [84]:
# Fill remaining NaNs in reviews_per_month with 0
df['reviews_per_month'] = df['reviews_per_month'].fillna(0)


In [87]:
df['survived'].sum()

9105

In [88]:
df.to_csv("../data/cleaned_austin_v1.csv", index=False)